# Data Reports

use pandas-profiling to generate data reports.

In [6]:
import os
from tqdm import tqdm
import warnings
from camelsp import Bundesland

First load all NUTS folders found in the output dir

In [7]:
nuts = [nt for nt in os.listdir(Bundesland('Hessen').base_path) if nt.startswith('DE')]
nuts

['DEF',
 'DED',
 'DE9',
 'DEG',
 'DE1',
 'DEE',
 'DE4',
 'DEC',
 'DE8',
 'DEB',
 'DE2',
 'DE7',
 'DEA']

Und jetzt gib ihm.

In [5]:
# set to true, if new output data was added (ie. rainfall)
REPLACE = False

# create for each report
for ID in nuts:
    with Bundesland(ID) as bl:
        with warnings.catch_warnings(record=True) as warn:
            nuts_ids = bl.nuts_table.nuts_id.values.tolist()

            for nuts_id in tqdm(nuts_ids, desc=ID):
                # write the report files
                try:
                    bl.generate_reports(nuts_ids=nuts_id, fmt='html', if_exists='replace' if REPLACE else 'omit')
                    bl.generate_reports(nuts_ids=nuts_id, fmt='json', if_exists='replace' if REPLACE else 'omit')
                except Exception as e:
                    print(str(e))
                    warnings.warn(str(e))

            if len(warn) > 0:
                print(f"FutureWarnings: {len([w for w in warn if w.category == FutureWarning])}")
                print(f"Other warnings: {len([w for w in warn if w.category != FutureWarning])} (possibly missing data files).")

100%|██████████| 282/282 [11:20<00:00,  2.41s/it]


There were 372 warnings (missing data files).


  2%|▏         | 1/63 [00:02<02:54,  2.82s/it]

DataFrame is empty. Please provide a non-empty DataFrame.


 14%|█▍        | 9/63 [00:20<02:02,  2.27s/it]

...

 79%|███████▉  | 429/540 [15:26<04:09,  2.25s/it]

DataFrame is empty. Please provide a non-empty DataFrame.


 88%|████████▊ | 473/540 [17:02<02:55,  2.62s/it]

: 

: 

In [8]:
# set to true, if new output data was added (ie. rainfall)
REPLACE = True

ID = 'DEB'

# create report for one bl
with Bundesland(ID) as bl:
    with warnings.catch_warnings(record=True) as warn:
        nuts_ids = bl.nuts_table.nuts_id.values.tolist()

        for nuts_id in tqdm(nuts_ids, desc=ID):
            # write the report files
            try:
                bl.generate_reports(nuts_ids=nuts_id, fmt='html', if_exists='replace' if REPLACE else 'omit')
                bl.generate_reports(nuts_ids=nuts_id, fmt='json', if_exists='replace' if REPLACE else 'omit')
            except Exception as e:
                print(str(e))
                warnings.warn(str(e))

        if len(warn) > 0:
            print(f"FutureWarnings: {len([w for w in warn if w.category == FutureWarning])}")
            print(f"Other warnings: {len([w for w in warn if w.category != FutureWarning])} (possibly missing data files).")

DEB:   2%|▏         | 3/155 [00:10<08:21,  3.30s/it]

DataFrame is empty. Please provide a non-empty DataFrame.
DataFrame is empty. Please provide a non-empty DataFrame.


DEB:   4%|▍         | 6/155 [00:13<04:26,  1.79s/it]

DataFrame is empty. Please provide a non-empty DataFrame.


DEB:   5%|▌         | 8/155 [00:16<04:20,  1.78s/it]

...

DataFrame is empty. Please provide a non-empty DataFrame.


DEB:  95%|█████████▍| 147/155 [05:34<00:21,  2.66s/it]

DataFrame is empty. Please provide a non-empty DataFrame.
DataFrame is empty. Please provide a non-empty DataFrame.
DataFrame is empty. Please provide a non-empty DataFrame.
DataFrame is empty. Please provide a non-empty DataFrame.
DataFrame is empty. Please provide a non-empty DataFrame.
DataFrame is empty. Please provide a non-empty DataFrame.
DataFrame is empty. Please provide a non-empty DataFrame.


DEB: 100%|██████████| 155/155 [05:37<00:00,  2.18s/it]

FutureWarnings: 465
Other warnings: 31 (possibly missing data files).
